# Logistic Regression Implementation from Scratch

## Introduction

Logistic regression is one of the most fundamental machine learning models for binary classification. I will summarize its methodology and implement it from scratch using NumPy.

### Binary classification

For example, the doctor would like to base on patients's features, including mean radius, mean texture, etc, to classify  breat cancer into one of the following two case: 
- "malignant": $y = 1$
- "benign": $y = 0$

which correspond to serious and gentle case respectively. 

We would like to load the breast cancer data from scikit-learn as a toy dataset, and split the data into the training and test datasets.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import itertools

import numpy as np
import scipy as sp
import pandas as pd

In [2]:
from sklearn.datasets import load_breast_cancer
bc_data = load_breast_cancer()

In [3]:
RANDOM_SEED = 71
TRAIN_PERCENT = 0.8

In [4]:
features = bc_data.get('feature_names')
features = ['_'.join(x.split()) for x in features]
X = bc_data.get('data')
X = X.reshape((X.shape[1], X.shape[0]))

print('feature_names: \n{}'.format(features))
print('X: \n{}'.format(X))

print('X.shape: {}'.format(X.shape))

feature_names: 
['mean_radius', 'mean_texture', 'mean_perimeter', 'mean_area', 'mean_smoothness', 'mean_compactness', 'mean_concavity', 'mean_concave_points', 'mean_symmetry', 'mean_fractal_dimension', 'radius_error', 'texture_error', 'perimeter_error', 'area_error', 'smoothness_error', 'compactness_error', 'concavity_error', 'concave_points_error', 'symmetry_error', 'fractal_dimension_error', 'worst_radius', 'worst_texture', 'worst_perimeter', 'worst_area', 'worst_smoothness', 'worst_compactness', 'worst_concavity', 'worst_concave_points', 'worst_symmetry', 'worst_fractal_dimension']
X: 
[[  1.79900000e+01   1.03800000e+01   1.22800000e+02 ...,   5.37200000e-01
    2.38800000e-01   2.76800000e-01]
 [  7.61500000e-02   1.35400000e+01   1.43600000e+01 ...,   4.61900000e-02
    4.83300000e-02   5.01300000e-02]
 [  1.98700000e-01   6.16900000e-02   1.49900000e+01 ...,   1.62400000e-01
    3.51100000e-01   3.87900000e-01]
 ..., 
 [  5.56700000e+02   1.10600000e-01   1.46900000e-01 ...,   7

In [5]:
target = bc_data.get('target_names')
Y = bc_data.get('target')
Y = Y.reshape((1, Y.shape[0]))

print('target_names: {}'.format(target))
print('target: \n{}'.format(Y))

print('Y: {}'.format(Y.shape))

target_names: ['malignant' 'benign']
target: 
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
  1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
  1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
  1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
  1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
  1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
  1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
  1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
  0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
  1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1
  0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 

We perform basic EDA for the breast cancer data.

In [6]:
# EDA for numbers of malignant and benign.
print('Number of malignant: {}'.format((Y == 0).sum()))
print('Number of benign: {}'.format((Y == 1).sum()))

Number of malignant: 212
Number of benign: 357


In [7]:
# EDA for feature matrix.
pd.DataFrame(X.T, columns=features).describe()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_radius,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,82.418324,80.149257,60.928757,50.710758,72.551373,50.093678,66.625860,51.650359,65.675619,62.371637,...,52.327407,50.158185,52.939044,62.088177,61.756132,59.385983,59.984329,57.359308,49.329894,59.330395
std,284.315843,281.218327,213.040592,182.652881,256.071418,188.256875,237.277474,173.442381,243.700592,249.772254,...,184.718843,165.364693,184.089029,214.003553,262.632038,198.181448,226.077133,216.440465,176.069135,222.413865
min,0.001997,0.001777,0.001286,0.000692,0.001435,0.000000,0.001465,0.000000,0.001519,0.000000,...,0.000000,0.001432,0.001180,0.001638,0.000000,0.001219,0.001541,0.001217,0.000000,0.000000
25%,0.078640,0.065400,0.055040,0.056160,0.065560,0.056770,0.061940,0.056280,0.053520,0.039900,...,0.055500,0.046050,0.052440,0.055810,0.052050,0.053550,0.059280,0.053850,0.045310,0.048240
50%,0.241600,0.225200,0.189500,0.181900,0.202700,0.192200,0.200100,0.169200,0.182200,0.166700,...,0.182500,0.162200,0.183600,0.165000,0.168700,0.177500,0.187000,0.170900,0.162800,0.171000
75%,18.250000,17.020000,15.530000,14.250000,16.150000,13.290000,15.760000,13.930000,14.770000,14.470000,...,13.720000,13.800000,13.980000,15.660000,14.920000,14.200000,14.690000,13.660000,13.870000,16.600000
max,2398.000000,2615.000000,2145.000000,1866.000000,2562.000000,2360.000000,2073.000000,1590.000000,2403.000000,3216.000000,...,2081.000000,1349.000000,1748.000000,2022.000000,4254.000000,1740.000000,2782.000000,2642.000000,1809.000000,2027.000000


In [8]:
def normalize_feature(x, axis=1):
    """Implement a function that normalizes each col or row of the matrix x 
    to have unit length.
    
    Args:
      x: A numpy matrix of shape (n, m).
      axis: A integer in {0, 1}, 
        - 0: normalize for each feature col.
        - 1: normalize for each feature row. 
    
    Returns:
      x_normalized: The normalized (by row) numpy matrix.
    """
    # Compute x_norm as the norm 2 of x.
    x_norm = np.linalg.norm(x, axis=axis, ord=2, keepdims=True)
    # Divide x by its norm.
    x_normalized = x / x_norm
    return x_normalized

In [9]:
X = normalize_feature(X)

print('Normalized X: {}'.format(X))
print('Normalized X.shape: {}'.format(X.shape))

# EDA for normalized feature matrix.
pd.DataFrame(X.T, columns=features).describe()

Normalized X: [[  2.54979475e-03   1.47119897e-03   1.74049358e-02 ...,   7.61395074e-05
    3.38460804e-05   3.92319725e-05]
 [  1.09260839e-05   1.94273377e-03   2.06038825e-03 ...,   6.62739090e-06
    6.93444040e-06   7.19270634e-06]
 [  3.76235879e-05   1.16809217e-05   2.83833711e-03 ...,   3.07502299e-05
    6.64803308e-05   7.34483632e-05]
 ..., 
 [  1.04314500e-01   2.07242386e-05   2.75261361e-05 ...,   1.45350741e-05
    2.18672572e-03   3.75134953e-03]
 [  1.72576186e-02   9.55008756e-02   2.33130441e-05 ...,   7.01915373e-05
    1.74985506e-05   2.49192578e-03]
 [  3.91512700e-03   1.24872137e-02   6.57077880e-02 ...,   0.00000000e+00
    5.23292813e-05   1.28298785e-05]]
Normalized X.shape: (30, 569)


,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_radius,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension
count,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,569.000000,5.690000e+02,569.000000,5.690000e+02,569.000000,...,569.000000,5.690000e+02,5.690000e+02,5.690000e+02,569.000000,5.690000e+02,5.690000e+02,5.690000e+02,569.000000,569.000000
mean,1.168148e-02,1.149990e-02,1.153678e-02,1.122401e-02,1.143708e-02,0.010789,1.134239e-02,0.011975,1.091750e-02,0.010165,...,0.011435,1.217814e-02,1.159554e-02,1.169054e-02,0.009604,1.204324e-02,1.075992e-02,1.074799e-02,0.011319,0.010814
std,4.029722e-02,4.034951e-02,4.033896e-02,4.042726e-02,4.036739e-02,0.040546,4.039415e-02,0.040211,4.051125e-02,0.040707,...,0.040368,4.014966e-02,4.032208e-02,4.029459e-02,0.040843,4.019040e-02,4.055347e-02,4.055664e-02,0.040401,0.040539
min,2.830428e-07,2.549659e-07,2.435024e-07,1.531630e-07,2.262150e-07,0.000000,2.494018e-07,0.000000,2.525090e-07,0.000000,...,0.000000,3.476819e-07,2.584622e-07,3.084180e-07,0.000000,2.472083e-07,2.764229e-07,2.280416e-07,0.000000,0.000000
25%,1.114596e-05,9.383662e-06,1.042175e-05,1.243011e-05,1.033495e-05,0.000012,1.054467e-05,0.000013,8.896828e-06,0.000007,...,0.000012,1.118069e-05,1.148623e-05,1.050843e-05,0.000008,1.085973e-05,1.063358e-05,1.009042e-05,0.000010,0.000009
50%,3.424294e-05,3.231194e-05,3.588158e-05,4.026062e-05,3.195386e-05,0.000041,3.406505e-05,0.000039,3.028778e-05,0.000027,...,0.000040,3.938129e-05,4.021496e-05,3.106774e-05,0.000026,3.599629e-05,3.354385e-05,3.202326e-05,0.000037,0.000031
75%,2.586646e-03,2.442048e-03,2.940585e-03,3.154007e-03,2.545904e-03,0.002862,2.682984e-03,0.003230,2.455272e-03,0.002358,...,0.002998,3.350566e-03,3.062120e-03,2.948611e-03,0.002320,2.879703e-03,2.635076e-03,2.559612e-03,0.003183,0.003026
max,3.398781e-01,3.752030e-01,4.061530e-01,4.130089e-01,4.038766e-01,0.508284,3.529078e-01,0.368626,3.994596e-01,0.524131,...,0.454775,3.275300e-01,3.828745e-01,3.807211e-01,0.661560,3.528650e-01,4.990321e-01,4.950582e-01,0.415092,0.369458


In [10]:
np.random.seed(RANDOM_SEED)
train_flag = np.random.rand(X.shape[1]) < TRAIN_PERCENT

X_train = X[:, train_flag]
Y_train = Y[:, train_flag]
X_test = X[:, ~train_flag]
Y_test = Y[:, ~train_flag]

print('X_train.shape: {}'.format(X_train.shape))
print('Y_train.shape: {}'.format(Y_train.shape))
print('X_test.shape: {}'.format(X_test.shape))
print('Y_test.shape: {}'.format(Y_test.shape))

X_train.shape: (30, 455)
Y_train.shape: (1, 455)
X_test.shape: (30, 114)
Y_test.shape: (1, 114)


## Methodology

TBD

## Implementation

In [11]:
# Sigmoid function.
def sigmoid(x):
    """Compute the sigmoid of x.

    Args:
      x: A scalar or numpy array of any size.

    Returns:
      s: sigmoid(x).
    """
    s = 1 / (1 + np.exp(-x))    
    return s

In [12]:
def sigmoid_derivative(s):
    """Compute the gradient of the sigmoid function.
    
    Args:
      s: A scalar or numpy array. Sigmoid function.

    Returns:
      ds: Computed gradient.
    """
    ds = s * (1 - s)    
    return ds

In [13]:
def initialize_weights(dim):
    """Initialize weights.

    This function creates a vector of zeros of shape (dim, 1) for w and b to 0.
    
    Args:
      dim: A integer. Size of the w vector (or number of parameters.)
    
    Returns:
      w: A Numpy array. Initialized vector of shape (dim, 1)
      b: A integer. Initialized scalar (corresponds to the bias)
    """
    w = np.zeros(dim).reshape(dim, 1)
    b = 0
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    return w, b

In [14]:
def propagate(w, b, X, Y):
    """Forward & backward propagation.

    Implement the cost function and its gradient for the propagation.

    Args:
      w: A Numpy array. Weights of size (num_px * num_px * 3, 1)
      b: A float. Bias.
      X: A Numpy array. Data of size (num_px * num_px * 3, number of examples).
      Y: A Numpy array. True "label" vector (containing 0 or 1) 
         of size (1, number of examples).

    Returns:
      cost: A float. Negative log-likelihood cost for logistic regression.
      dw: A Numpy array. Gradient of the loss w.r.t. w, thus same shape as w.
      db: A float. Gradient of the loss w.r.t b, thus same shape as b.
    """
    m = X.shape[1]
    
    # Forward propagation from X to cost.
    # Compute activation.
    A = sigmoid(np.dot(w.T, X) + b)
    # Compute cost.
    cost = - 1 / m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))
    
    # Backward propagation to find gradient.
    dw = 1 / m * np.dot(X, (A - Y).T)
    db = 1 / m * np.sum(A - Y)

    assert(dw.shape == w.shape)
    assert(db.dtype == float)

    cost = np.squeeze(cost)
    assert(cost.shape == ())

    grads = {"dw": dw,
             "db": db} 

    return grads, cost

In [15]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost=True):
    """Optimization function.

    This function optimizes w and b by running a gradient descent algorithm.
    That is, write down two steps and iterate through them:
      1. Calculate the cost and the gradient for the current parameters. 
        Use propagate().
      2. Update the parameters using gradient descent rule for w and b.
    
    Args:
      w: A Numpy array. Weights of size (num_px * num_px * 3, 1).
      b: A scalar. Bias.
      X: A Numpy array. Data of shape (num_px * num_px * 3, number of examples).
      Y: A Numpy array. True "label" vector (containing 0 if non-cat, 1 if cat), 
        of shape (1, number of examples)
      num_iterations: A integer. Number of iterations of the optimization loop.
      learning_rate: A scalr. Learning rate of the gradient descent update rule.
      print_cost: A Boolean. Print the loss every 100 steps. Default: True.
    
    Returns:
      params: A dictionary containing the weights w and bias b.
      grads: A dictionary containing the gradients of the weights and bias 
        with respect to the cost function
      costs: A list of all the costs computed during the optimization, 
        this will be used to plot the learning curve.
    """   
    costs = []

    for i in range(num_iterations):
        # Cost and gradient calculation (≈ 1-4 lines of code)
        grads, cost = propagate(w, b, X, Y)
        
        # Retrieve derivatives from grads
        dw = grads.get('dw')
        db = grads.get('db')
        
        # Update rule.
        w -= learning_rate * dw
        b -= learning_rate * db
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [16]:
def predict(w, b, X):
    """Prediction.

    Predict whether the label is 0 or 1 using learned logistic regression 
    parameters (w, b)
    
    Args:
      w: A Numpy array. Learned weights of size (num_px * num_px * 3, 1).
      b: A scalar. Learned bias.
      X: A Numpy array. New data of size (num_px * num_px * 3, number of examples).
    
    Returns:
      Y_prediction: A Numpy array containing all predictions (0/1) 
        for the examples in X.
    """
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a label 1 
    # being present in the picture.
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        # Convert probabilities a[0,i] to actual predictions p[0,i]
        if A[0, i] > 0.5:
            Y_prediction[0, i] = 1
        else:
            Y_prediction[0, i] = 0
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [17]:
def accuracy(Y_prediction, Y):
    acc = 1 - np.mean(np.abs(Y_prediction - Y))
    return acc

In [18]:
def logistic_regression(X_train, Y_train, X_test, Y_test, 
                        num_iterations=2000, learning_rate=0.001, print_cost=True):
    '''Wrap-up function for logistic regression.

    Builds the logistic regression model by calling the function 
    you've implemented previously.
    
    Args:
      X_train: A Numpy. Training set of shape (num_px * num_px * 3, m_train).
      Y_train: A Numpy array. Training labels of shape (1, m_train).
      X_test: A Numpy array. Test set of shape (num_px * num_px * 3, m_test).
      Y_test: A Numpy array. Test labels of shape (1, m_test).
      num_iterations: An integer. Hyperparameter for the number of iterations 
        to optimize the parameters. Default: 2000.
      learning_rate: A scalar. Hyperparameter for the learning rate used 
        in the update rule of optimize(). Default: 0.005.
      print_cost: A Boolean. Print the cost every 100 iterations. Default: True.
    
    Returns:
      d: A dictionary containing information about the model.
    '''    
    # initialize parameters with zeros (≈ 1 line of code)
    w, b = initialize_weights(X_train.shape[0])

    # Gradient descent.
    parameters, grads, costs = optimize(
        w, b, X_train, Y_train, 
        num_iterations=num_iterations, learning_rate=learning_rate, 
        print_cost=print_cost)
    
    # Retrieve parameters w and b from dictionary 'parameters'
    w = parameters.get('w')
    b = parameters.get('b')
    
    # Predict test/train set examples (≈ 2 lines of code)
    Y_pred_train = predict(w, b, X_train)
    Y_pred_test = predict(w, b, X_test)

    # Print train/test Errors
    print('Train accuracy: {} %'
          .format(accuracy(Y_pred_train, Y_train) * 100))
    print('Test accuracy: {} %'
          .format(accuracy(Y_pred_test, Y_test) * 100))
    
    d = {'costs': costs,
         'Y_pred_train': Y_pred_train, 
         'Y_pred_test': Y_pred_test, 
         'w': w, 
         'b': b,
         'learning_rate' : learning_rate,
         'num_iterations': num_iterations}
    return d

In [19]:
d = logistic_regression(X_train, Y_train, X_test, Y_test)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.691354
Cost after iteration 200: 0.689648
Cost after iteration 300: 0.688026
Cost after iteration 400: 0.686482
Cost after iteration 500: 0.685015
Cost after iteration 600: 0.683618
Cost after iteration 700: 0.682290
Cost after iteration 800: 0.681027
Cost after iteration 900: 0.679825
Cost after iteration 1000: 0.678681
Cost after iteration 1100: 0.677594
Cost after iteration 1200: 0.676559
Cost after iteration 1300: 0.675574
Cost after iteration 1400: 0.674637
Cost after iteration 1500: 0.673746
Cost after iteration 1600: 0.672897
Cost after iteration 1700: 0.672090
Cost after iteration 1800: 0.671322
Cost after iteration 1900: 0.670590
Train accuracy: 63.5164835165 %
Test accuracy: 59.649122807 %


In [31]:
from sklearn.linear_model import LogisticRegression

logist_reg = LogisticRegression(C=10**6, max_iter=2000)
logist_reg.fit(X_train.T, Y_train.flatten())
Y_pred_test = logist_reg.predict(X_test.T)

print('Test accuracy: {} %'
      .format(accuracy(Y_pred_test, Y_test.flatten()) * 100))

Test accuracy: 53.5087719298 %
